# DAT 19: Homework 2 Assignment

## Instructions

For Homework 2, we will build on the work we did with the Titanic dataset in Homework 1. In this assignment, we will build a logistic regression model to predict passenger survival.

Please do all your analysis to answer the questions below in this Jupyter notebook. Show your work.

**Please submit your completed notebook by 6:00PM on Monday, January 11.**

## About the Data

```
VARIABLE DESCRIPTIONS:
survival        Survival
                (0 = No; 1 = Yes)
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.
```

## Homework Assignment

In [74]:
import numpy as np
import csv  as csv
import pandas as pd
from sklearn.cross_validation import cross_val_score
from sklearn import linear_model, metrics

In [75]:
% ls

 Volume in drive C is Windows7_OS
 Volume Serial Number is 2ED8-D2F5

 Directory of C:\Users\Nermin\DAT_SF_19\hw-assignments

01/20/2016  01:11 PM    <DIR>          .
01/20/2016  01:11 PM    <DIR>          ..
01/19/2016  12:12 PM    <DIR>          .ipynb_checkpoints
01/19/2016  11:19 AM           120,913 DAT19-HW1-Assignment.ipynb
01/19/2016  12:10 PM            19,267 DAT19-HW1-Solutions.ipynb
01/20/2016  01:11 PM            55,661 DAT19-HW2-Assignment.ipynb
01/13/2016  06:42 PM             6,152 DAT19-HW3-Assignment.ipynb
12/14/2015  06:43 PM            61,194 titanic.csv
01/04/2016  06:49 PM            28,627 titanic-test.csv
01/20/2016  11:03 AM            12,405 Untitled1.ipynb
               7 File(s)        304,219 bytes
               3 Dir(s)  366,602,964,992 bytes free


In [99]:
# Reading in the train and test datasets
data = pd.read_csv('C:\\Users\\Nermin\\DAT_SF_19\\hw-assignments\\titanic.csv')
test_data = pd.read_csv('C:\\Users\\Nermin\\DAT_SF_19\\hw-assignments\\titanic-test.csv')
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [100]:
# Checking for missing values
print data.isnull().values.any() # null values 
print test_data.isnull().values.any() # null values exist

True
True


In [101]:
# Remove empty (null) cells
data = data.dropna()
test_data = test_data.dropna()

In [102]:
# Taking out name, ticket number, and cabin features, as they are useless predictor features for the model
data = data.drop(['Name','Ticket','Cabin'],axis=1) # axis 0 refers the rows, 1 refers to columns
test_data = test_data.drop(['Name','Ticket','Cabin'],axis=1) # axis 0 refers the rows, 1 refers to columns

In [103]:
# creating One-hot-encoding (a.k.a. -- indicator, dummy, binary) features
categorical_feats = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']
# For every categorical variable in train data
for feat in categorical_feats:
    # Create all indicator variables for the categorical variable
    data_indicators = pd.get_dummies(data[feat], prefix=feat)
    # Drop the categorical variable from data, since we have indicator variables now
    data = data.drop([feat], axis=1)
    # Joining the original data with the newly created indicator variables
    data = pd.concat((data, data_indicators.ix[:,1:]), axis=1)
# For every categorical variable in test data
for feat in categorical_feats:
    # Create all indicator variables for the categorical variable
    test_data_indicators = pd.get_dummies(test_data[feat], prefix=feat)
    # Drop the categorical variable from data, since we have indicator variables now
    test_data = test_data.drop([feat], axis=1)
    # Joining the original data with the newly created indicator variables
    test_data = pd.concat((test_data, test_data_indicators.ix[:,1:]), axis=1)

In [104]:
# checking that test and train sets have all of the same indicator variables
print test_data.columns[1:]
print data.columns[2:]

Index([u'Age', u'Fare', u'Pclass_2', u'Pclass_3', u'Sex_male', u'SibSp_1',
       u'SibSp_2', u'SibSp_3', u'Parch_1', u'Parch_2', u'Parch_3', u'Parch_4',
       u'Embarked_Q', u'Embarked_S'],
      dtype='object')
Index([u'Age', u'Fare', u'Pclass_2', u'Pclass_3', u'Sex_male', u'SibSp_1',
       u'SibSp_2', u'SibSp_3', u'Parch_1', u'Parch_2', u'Parch_4',
       u'Embarked_Q', u'Embarked_S'],
      dtype='object')


In [106]:
# Removing indicator feature from test set that training set does have 
test_data = test_data.drop(['Parch_3'], axis=1)

In [107]:
### Need to turn into a numpy array first
data_array = data.as_matrix()
test_data_array = test_data.as_matrix()

**1) Create a logistic regression model on the Titanic dataset to predict the survival of passengers. Show your model output. Include coefficient values.**

In [108]:
X = data_array[:, 2:]  # Getting predictor variables or features
Y = data_array[:, 1] # Getting second column, the target feature... survived or not
X_test = test_data_array[:, 1:] # get rid of ID column

## Y = x1, x2, ... xn what the logistic regression equation looks like

# Specifying which machine learning algorithm to use
logreg = linear_model.LogisticRegression() 

# fitting the data to the logistic regression model object
logreg.fit(X, Y)

#now we have logistic regression model y = x1 + x2 + xm
# where y = ln(P/1-P) and P = probability that the example is of class 1



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [109]:
# getting predictor feature coefficients from the model
print logreg.coef_.ravel()

[-0.01858103  0.00350639  0.39609277 -0.55637466 -2.06342251  0.41747724
  0.21893953 -0.3359818  -0.08825063 -0.23207253 -0.46381487 -0.30338761
 -0.06201527]


**2) Which features are predictive for this logistic regression? Explain your thinking. Do not simply cite model statistics.**

In [110]:
print 'The Variables with the most predictive power are as follows:'
import_vars = data.columns[[5,6,9],]
for var in import_vars:
    print str(var)

The Variables with the most predictive power are as follows:
Pclass_3
Sex_male
SibSp_3


These variables have the most predictive power because they have the most extreme coeffient (Beta) values. It intuitively makes sense that the Male and Passenger Class 3 variables are two of the most predictive, as in the previous homework it was shown that the survival rate depended heavily on which sex and which passenger class the passenger was. 

**3) Implement cross-validation for your logistic regression model. Select the number of folds. Explain your choice.**

In [111]:
cross_val_score(logreg,X,Y,cv=10).mean()

0.72076023391812849

I chose to do a 10-fold cross-validation, as it is the industry standard and in this case was not too computationally expensive (slow).

**4) In the hw-assignments director on the class github repo, there is a file called titanic-test.csv. What does your logistic regression model predict for these previously unseen (i.e. out of sample) passengers?**

In [113]:
# use model to predict test set
predicted = logreg.predict(X_test)
print predicted

[ 1.  1.  1.  1.  0.  1.  1.  0.  1.  1.  1.  0.  1.  1.  0.  1.  0.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  1.  1.  1.  0.  0.
  0.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.  1.  1.  1.  1.
  0.  1.  1.  0.  1.  1.  0.  1.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.
  1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.]
